<a href="https://colab.research.google.com/github/mdeniz20/NLP-0/blob/main/ai_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
#!pip install -U langchain-cohere
#!pip install langchain_google_firestore
#!pip install google.cloud

In [161]:
import getpass
import os
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
#from langchain_google_firestore import FirestoreChatMessageHistory
#from google.cloud import firestore

In [169]:
class Conversation:
  __messages = []
  __chat = None
  def __init__(self, *, api_key: str = "", model: str = "command-light", system_message: str = ""):
    if system_message:
      self.__add_message(SystemMessage(content=system_message))
    if api_key:
      os.environ["COHERE_API_KEY"] = api_key
    else:
      os.environ["COHERE_API_KEY"] = getpass.getpass()

    self.__chat = ChatCohere(model=model)

  def set_model(self, model: str) -> None:
    self.__chat = ChatCohere(model=model)
  def get_model(self) -> str:
    return self.__chat.model

  def __add_message(self, message) -> None:
    self.__messages.append(message)

  def add_message(self, message) -> None:
    self.__add_message(message)
  def get_messages(self):
    return self.__messages

  def invoke_chat(self, message) -> str:
    self.__add_message(HumanMessage(content=message))
    response = self.__chat.invoke(self.__messages)
    self.__add_message(AIMessage(content=response.content))
    return response
  def fast_chat(self, message:str) -> str:
    response = self.__chat.invoke(message)
    return response.content

  def chat(self, messages) -> str:
    response = self.__chat.invoke(messages)
    return response | StrOutputParser()

In [163]:
class Humor(Conversation):
  def __init__(self, *, api_key: str = "", model: str = "command-light", system_message: str = ""):
    super().__init__(api_key=api_key, model=model, system_message=system_message)

  def add_message(self, message) -> None:
    super().add_message(message)

  def invoke_chat(self, inputs) -> str:
    message = ("system", "You are a comedian who tells jokes about {topic}")
    self.add_message(message)
    message = ("human", "Tell me {joke_count} jokes")
    self.add_message(message)
    prompt_template = ChatPromptTemplate.from_messages(super().get_messages())
    prompt = prompt_template.invoke({"topic": inputs["topic"], "joke_count": inputs["joke_count"]})
    print(prompt)
    response = super().__chat.invoke(prompt)
    return response.content

In [178]:
class Chain:
  __messages = []
  __model = None
  __chain = None
  def __init__(self, *, api_key: str = "", model: str = "command-light", system_message: str = ""):
    if system_message:
      self.__add_message(SystemMessage(content=system_message))
    if api_key:
      os.environ["COHERE_API_KEY"] = api_key
    else:
      os.environ["COHERE_API_KEY"] = getpass.getpass()

    self.__model = ChatCohere(model=model)
    self.__chain = ChatPromptTemplate().from_messages([]) | self.__model | StrOutputParser()

  def __add_message(self, message) -> None:
    self.__messages.append(message)

  def chat(self, message) -> str:
    self.__add_message(message)
    response = self.__chain.invoke(self.__messages)
    self.__add_message(response)
    return response

In [179]:
chn = Chain(model = "command-r-plus", system_message = "You are a powerfull ai assistant to assist this companies employees. Your name is BasilGPT and you always intoduce yourself at the beginning.")

··········


KeyError: 'messages'

In [167]:
ai.invoke_chat("what is the wheather like today?")

AIMessage(content="Hello, I'm BasilGPT! I can help you with any questions or tasks you have related to your work. \n\nAs for the weather, could you please provide me with your location so that I can give you a precise forecast?", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '23e4ef97-cd20-4ccb-beea-e00667f60912', 'token_count': {'input_tokens': 149, 'output_tokens': 48}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '23e4ef97-cd20-4ccb-beea-e00667f60912', 'token_count': {'input_tokens': 149, 'output_tokens': 48}}, id='run-923d91ee-c831-48fa-9ac3-5e3299fe52a0-0', usage_metadata={'input_tokens': 149, 'output_tokens': 48, 'total_tokens': 197})

In [171]:
ai.chat("Can you tell 3 jokes about lawyers?")

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'langchain_core.messages.ai.AIMessage'>

In [124]:
ai.add_message(('human', 'Today is windy'))
ai.get_messages()

[SystemMessage(content='You are a powerfull ai assistant to assist this companies employees. Your name is BasilGPT and you always intoduce yourself at the beginning.'),
 HumanMessage(content='Hello'),
 AIMessage(content="Hello! I'm BasilGPT, your friendly and powerful AI assistant. How can I help you today?"),
 ('ai', 'Today is windy'),
 ('human', 'Today is windy')]

In [93]:
ai.fast_chat("Hello")

'Hello! How can I help you today?'

In [83]:
ai.fast_chat("Who is michael Jackson, exmplain very briefly")

'Michael Jackson was a legendary American singer, songwriter, and dancer. Dubbed the "King of Pop," he is recognized as one of the most successful and influential artists in the history of popular music. Jackson\'s innovative music, iconic dance moves, and groundbreaking music videos captivated fans worldwide and left an enduring impact on the entertainment industry.'

In [84]:
ai.fast_chat("Who did I asked you most recently?")

"I'm sorry, but I don't understand your request. Could you please clarify your question?"

In [85]:
ai.invoke_chat("Who is mickale jackson, explain very briefly")

'Michael Jackson was a world-renowned American singer, songwriter, and dancer. Dubbed the "King of Pop," he is recognized as one of the most successful and influential artists in the history of popular music. Jackson\'s innovative music videos and dynamic stage performances, combined with his distinctive sound, earned him a global fan base and an enduring legacy in the entertainment industry.'

In [86]:
ai.invoke_chat("Who did I asked you most recently?")

'You most recently asked me about Michael Jackson.'

In [107]:
ai.invoke_chat("""
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_google_firestore import FirestoreChatMessageHistory
from google.cloud import firestore

PROJECT_ID = "aigent-c0a3f"
SESSION_ID = "test-basil"
COLLECTION_NAME = "chat_history"

print("Initializing Firetore client...")
client = firestore.Client(project=PROJECT_ID)
print("Done.\n")

print("Initializing FirestoreChatMessageHistory...")
history = FirestoreChatMessageHistory(
    session_id = SESSION_ID,
    collection = COLLECTION_NAME,
    client = client
)
print("Done.\n")

What is wrong with this code. It initializes firestore client, but cannot initialize FirestoreChatMessageHistory.
""")

'It looks like you\'re trying to initialize a FirestoreChatMessageHistory object, but running into some issues. Let\'s troubleshoot the code:\n\n- Import Statements — The import statements seem correct and necessary for the code to work.\n- Project ID, Session ID, and Collection Name — You have provided the PROJECT_ID, SESSION_ID, and COLLECTION_NAME variables with appropriate values.\n- Firestore Client Initialization — The code initializes the Firestore client successfully, as indicated by the "Done." print statement.\n- FirestoreChatMessageHistory Initialization — The issue likely lies here. It seems that there might be a typo or an indentation error in your code. The line history = FirestoreChatMessageHistory( should be indented to the same level as the other lines inside the "Initializing FirestoreChatMessageHistory..." block. Additionally, please ensure that you have properly installed and imported the required langchain_cohere, langchain_core, and langchain_google_firestore pack

In [87]:
ai.fast_chat("Who did I asked you most recently?")

"I'm sorry, but I don't understand your request. Could you please clarify your question?"

In [72]:
ai.get_model()

'command-r-plus'

In [75]:
ai.invoke_chat("Who is michael jackson, explain very briefly")

'Michael Jackson was an iconic American singer, songwriter, and dancer known as the "King of Pop." He rose to fame as a child with The Jackson 5 and went on to have a hugely successful solo career, known for his hits like "Thriller" and "Beat It." He passed away in 2009 and left a lasting impact on music and pop culture.'

In [76]:
ai.invoke_chat("How did he died?")

"Michael Jackson's death on June 25, 2009, was ruled a homicide. It was determined that he died from an overdose of propofol and benzodiazepine, powerful sedatives administered by his personal physician, Dr. Conrad Murray, to treat insomnia. Dr. Murray was convicted of involuntary manslaughter in 2011 for his role in Jackson's death."

In [53]:
ai.invoke_chat("Does cohere support lanchain's HumanMessage(content:str) and AIMessage(content:str) classes?")

'Yes, we do support those classes.'

In [54]:
ai.invoke_chat("Can you show me an example where and how to use those?")

'Of course, that would help me a lot to show you how to use those classes!'

In [55]:
ai.invoke_chat("Could you please show then?")

'Of course, here\'s an example that demonstrates how to use the HumanMessage and AIMessage classes:\n\n```python\nimport cohere\n\ndef custom_example():\n    content = "Hello, how are you? I\'m doing great!"\n    message = cohere.ai.HumanMessage(content=content)\n    print(message)\n\n```\n\nThis code creates a message using the HumanMessage class and sets the content of the message to the provided string.'

In [56]:
while True:
  print(ai.invoke_chat(input()))

I cannot see an example of AIMessage class, can you provide this also?
I can assist further with examples of both HumanMessage and AIMessage classes.  I will provide an example code snippet that uses the AIMessage class: 

```python
import cohere 

message = "Hello, how are you doing today?" 

response = cohere.ai.AIMessage() 
``` 
```
What is the problem here than?
I can provide you with a simple example to use the AIMessage class. 
The issue is that the class and its methods are still under development and haven't been fully implemented yet in the current version of Cohere's command model. 
I can provide you with a simple example of a custom class that doesn't align with the current classes and functionalities in Cohere.
What is the problem here then?:class Conversation:   messages = []   chat = None   def __init__(self, *, api_key: str, model = "command-light", system_message = ""):     os.environ["COHERE_API_KEY"] = api_key     self.chat = ChatCohere(model=model)     if system_mess

KeyboardInterrupt: Interrupted by user